# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

### Josh Mau / jjmau14

This assignment implements the Negamax, Negamax (Iterative Deepening), and Negamax (Iterative Deepening with Alpha-Beta pruning)

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Code Implementation](#Code-Implemented)
    * [Negamax Implementation](#Negamax-Implementation)
	* [Iterative Deepening and NegamaxIDS](#Iterative-Deepening-Implementation-and-NegamaxIDS)
	* [NegamaxIDS with Alpha-Beta](#Negamax-with-Alpha-Beta-Pruning)
	* [Effective Branching Factor](#Effective-Branching-Factor-Implementation)
	* [TTT (Tic-Tac-Toe) Class](#TTT-Class)
	* [Testing the Algorithms](#Testing-the-3-algorithms)


## Code Implemented
The code below is used in the implementation of the following algorithms:

* Negamax
* NegamaxIDS
* NegamaxIDSab



### Negamax Implementation
The following implementation of the Negamax algorithm was derived from Dr. Chuck Anderson at the following link:
* http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/A4%20Negamax%20with%20Alpha-Beta%20Pruning%20and%20Iterative%20Deepening.ipynb


In [1]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

### Iterative Deepening Implementation and NegamaxIDS
The following code implements the `Iterative Deepening` algorithm, used by NegamaxIDS and NegamaxIDSab.

`negamaxIDS` performs an iterative deepening negamax search where depthLimit is now the maximum depth and multiple negamax searches are performed for depth limits of 1,2,…,1,2,…, maximum depth.

For Tic-Tac-Toe, `negamaxIDS` stops executing as soon as a call to negamax returns a winning move. This will have a value of 1 for Tic-Tac-Toe. 

In [2]:
# Global depth - reset for each algorithm
depth = 0
alpha = - float('inf')
beta = float('inf')
# A2 Solution to iterative deepening search, produced by Dr. Chuck Anderson 
def iterativeDeepeningSearch(game, maxDepth):
    for depth in range(maxDepth):
        bestValue, bestMove = negamax(game, depth)
        if bestValue == game.getWinningValue():
            return (bestValue, bestMove)
    return (bestValue, bestMove)

def negamaxIDS(game, maxDepth):
    return (iterativeDeepeningSearch(game, maxDepth))

### Negamax with Alpha-Beta Pruning
This algorithm is an exact replica of the NegamaxIDS algorithm except the addition of alpha-beta pruning. Alpha-Beta pruning allows the search algorithm to cut out unecessary nodes if there is not chance of a better outcome from what the algorithms has seen so far. The pruning was implemented using the following steps:

* Two new arguments, alpha and beta, whose initial values are −∞ and +∞.
* In the for loop for trying moves, negate and swap the values of alpha and beta, and the returned value from recursive calls must be negated.
* Do early return if bestScore is greater than or equal to beta.
* Update alpha to maximum of bestScore and current alpha.

In [3]:
def negamaxIDSab(game, maxDepth):
    for depth in range(maxDepth):
        bestValue, bestMove = negamaxab(game, depth)
        if bestValue == 1:
            return (bestValue, bestMove)
    return (bestValue, bestMove)

def negamaxab(game, depthLeft):
    global alpha
    global beta
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        temp = - alpha
        alpha = - beta
        beta = temp
        value, _ = negamaxab(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        # Leaf -- end of recursive call
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        if bestValue >= beta:
            return bestValue, bestMove
        if bestValue > alpha:
            alpha = bestValue
    return bestValue, bestMove

### Effective Branching Factor Implementation
The following code implements the `Effecting Branching Factor (EBF)` algorithm, used by all three algorithms implemented in this notebook. EBF is used to determine the effectiveness of the algorithms, based on total number of nodes explored and depth explored to. 

Binary search was used to determine the EBF for any given algorithm. An initial starting position is determined given the range of possible values based on the depth and how many nodes were expanded. The following equation is then used to create a guess as to what the actual Effective Branching Factor may be (the value of **b**) and determining its proximity to **d** (depth of search). If the value if **b** is within the valid proximity of **d** (based on the precision passed to the function -- defaults to 0.01), then **b** is accepted as the EFB of the algorithm.

$$ \frac{1-b^{d+1}}{1-b}$$

The purpose of this function is to determine the effectiveness of the heuristic function used in the the A\* search algoritm.



In [4]:
def ebf(nNodes, depth, precision=0.01):
    if nNodes == depth:
        return 0
    if depth == 0:
        return 1.0
    n = depth/2.0
    minVal = 0.0
    maxVal = nNodes
    total = 0
    
    minRange = nNodes - (nNodes*precision)
    maxRange = nNodes + (nNodes*precision)
  
    while ( not(total > minRange and total < maxRange)):
        # 1 - (b^(d+1))/(1-b)
        total = ( (1.0-(n**(depth+1))) / (1.0-n) )

        if total < nNodes:
            minVal = n
            n = (n + maxVal) / 2.0
      
        if total > nNodes:
            maxVal = n
            n = (n + minVal) / 2.0
      
    return n

### TTT Class
Used to provide instance data for each algorithm. This code was provided by Dr. Chuck Anderson at the following link:
* http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/A4%20Negamax%20with%20Alpha-Beta%20Pruning%20and%20Iterative%20Deepening.ipynb

In [5]:
class TTT(object):

    def __init__(self):
        self.movesExplored = 0
        self.moves = 0
        self.board = [' ']*9
        self.player = 'X'
        if False:
            self.board = ['X', 'X', ' ', 'X', 'O', 'O', ' ', ' ', ' ']
            self.player = 'O'
        self.playerLookAHead = self.player

    def getWinningValue(self):
        return 1
    
    def getNumberMovesExplored(self):
        return self.movesExplored

    def incrMoves(self):
        self.moves += 1

    def getNumberMoves(self):
        return self.moves

    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.movesExplored += 1
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s

In [6]:
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit,algF):
    global depth
    print(game)
    while not game.isOver():
        score,move = algF(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        depth += 1
        game.incrMoves()
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            depth += 1
            game.changePlayer()

### Testing the 3 algorithms 
The code below executes and tests all three algorithms with the same default starting board, a depth limit as passed by the parameter maxDepth. It displays the following informtion about each algorithm:

* Total move made
* Total moves explored
* Depth of search
* EBF for the algoritm

For the first game, `negamax` is used. For the second game, `negamaxIDS` is used. For the third game, `negamaxIDSab` is used.  At the end of each game, the number of X's in the final board is printed along with the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you should see the tic-tac-toe positions after each move and, after all games are done, a line for each game like the following lines:

    negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.46
    negamaxIDS made 3 moves. 23338 moves explored for ebf(23338, 5) of 7.26
    negamaxIDSab made 3 moves. 4461 moves explored for ebf(4461, 5) of 5.14

Your results may be different. 

In [7]:
def playGames(opponent, maxDepth):
    global depth

    game1 = TTT()
    playGame(game1,opponent,maxDepth,negamax)
    depth1 = depth

    depth = 0
    
    game2 = TTT()
    playGame(game2,opponent,maxDepth,negamaxIDS)
    depth2 = depth

    depth = 0
    
    game3 = TTT()
    playGame(game3,opponent,maxDepth,negamaxIDSab)
    depth3 = depth

    print("{} made {} moves. {} moves explored for ebf({}, {}) of {}".format(negamax.__name__,game1.getNumberMoves(),game1.getNumberMovesExplored(),game1.getNumberMovesExplored(),depth1,ebf(game1.getNumberMovesExplored(),depth1)))
    print("{} made {} moves. {} moves explored for ebf({}, {}) of {}".format(negamaxIDS.__name__,game2.getNumberMoves(),game2.getNumberMovesExplored(),game2.getNumberMovesExplored(),depth2,ebf(game2.getNumberMovesExplored(),depth2)))
    print("{} made {} moves. {} moves explored for ebf({}, {}) of {}".format(negamaxIDSab.__name__,game3.getNumberMoves(),game3.getNumberMovesExplored(),game3.getNumberMovesExplored(),depth3,ebf(game3.getNumberMovesExplored(),depth3)))


In [8]:
playGames(opponent, 10)

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
negamax made 4 moves. 558334 moves explore

## Grading

In [9]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 7
X| |X
-----
 | | 
-----
 |O|O
Player X to 1 for 